***ORIGNALLY FROM THEO VIEL'S BASELINE NOTEBOOK - https://www.kaggle.com/code/theoviel/evaluation-metric-folds-baseline/notebook***

# Intro

This notebook setups a naive solution for the problem, which achieves LB 0.568.

It consists of three main parts :
- Creating folds
- Building the evaluation metric
- Training & evaluation the baseline


Updates :
- v5 : lower strings before matching

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import ast
import numpy as np
import pandas as pd
from collections import Counter

pd.options.display.max_colwidth = 200

# Data

In [16]:
from project.data.data_loaders import get_clean_train_data
from sklearn.model_selection import GroupKFold
data = get_clean_train_data()
train_indices, test_indices = list(GroupKFold(n_splits=5).split(data['pn_history'], data['location'], data['pn_num']))[0]
train_data = data.iloc[train_indices]
valid_data = data.iloc[test_indices]

DATA_PATH = "../../../data/"
patient_notes = pd.read_csv(DATA_PATH + "patient_notes.csv")


In [17]:

train_data_grouped = train_data.groupby(['case_num', 'pn_num','pn_history'], as_index=False).agg(list)
patient_notes = train_data_grouped #patient_notes.merge(train_data_grouped, how="left", on=['case_num', 'pn_num'])

patient_notes = patient_notes.dropna(axis=0).reset_index(drop=True)
patient_notes = patient_notes[['case_num', 'pn_num', 'pn_history', 'annotation', 'location', 'feature_text', 'feature_num']]
patient_notes

,case_num,pn_num,pn_history,annotation,location,feature_text,feature_num
0,0,16,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[[dad with recent heart attcak], [mom with ""thyroid disease], [chest pressure], [intermittent episodes, episode], [felt as if he were going to pass out], [], [adderall, adderrall, adderrall], [], ...","[[(696, 724)], [(668, 693)], [(203, 217)], [(70, 91), (176, 183)], [(222, 258)], [], [(321, 329), (404, 413), (652, 661)], [], [], [(26, 38), (96, 118)], [(56, 69)], [(5, 9)], [(10, 11)]]","[Family-history-of-MI-OR-Family-history-of-myocardial-infarction, Family-history-of-thyroid-disorder, Chest-pressure, Intermittent-symptoms, Lightheaded, No-hair-changes-OR-no-nail-changes-OR-no-t...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,0,41,"17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING. STARTED 2-3 MO AGO. IT STARTED SUDDENLY. DOES NOT RECALL ANY TRIGGERING EVENTS. IT COMES AND GOES, IT HAPPENED 5-6 TIMES SINCE IT STARTED. IT LASTS ...","[[], [MOM HAS THYROID PROBLEMS], [PRESSURE ON HER CHEST], [COMES AND GOES, HAPPENED 5-6 TIMES], [], [], [], [SOB], [], [HEART POUNDING], [STARTED 2-3 MO AGO], [17 Y/O], [M]]","[[], [(532, 556)], [(263, 284)], [(131, 145), (150, 168)], [], [], [], [(258, 261)], [], [(32, 46)], [(48, 66)], [(0, 6)], [(7, 8)]]","[Family-history-of-MI-OR-Family-history-of-myocardial-infarction, Family-history-of-thyroid-disorder, Chest-pressure, Intermittent-symptoms, Lightheaded, No-hair-changes-OR-no-nail-changes-OR-no-t...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
2,0,46,Mr. Cleveland is a 17yo M who was consented by parents to be alone today and provide information. He has no notable PMH and presented today with palpitations that have been occurring sporadically...,"[[father: heart attack], [mom: thyroid issues], [], [sporadically], [light headedness], [has not had any cold/heat intolerance], [adderall, Adderall], [SOB], [], [palpitations, palpitations, palpi...","[[(824, 844)], [(803, 822)], [], [(184, 196)], [(552, 568)], [(582, 619)], [(380, 388), (730, 738)], [(573, 576)], [], [(146, 158), (436, 448), (478, 490)], [(197, 211)], [(19, 23)], [(24, 25)]]","[Family-history-of-MI-OR-Family-history-of-myocardial-infarction, Family-history-of-thyroid-disorder, Chest-pressure, Intermittent-symptoms, Lightheaded, No-hair-changes-OR-no-nail-changes-OR-no-t...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
3,0,100,"HPI: Dillon Cleveland is an otherwise healthy 17 year old male presenting due to rapid pounding heart rate for 2-3 months. He has never experienced these symptoms before. Comes on randomly, nothin...","[[Dad-MI], [Mom-thyroid problem], [chest pressure], [5-6 times], [light headedness], [], [adderall], [], [], [pounding heart, pounding heart], [2-3 months, 2-3 months], [17 year old], [male]]","[[(735, 741)], [(714, 733)], [(310, 324)], [(247, 256)], [(292, 308)], [], [(595, 603)], [], [], [(87, 101), (372, 386)], [(111, 121), (267, 277)], [(46, 57)], [(58, 62)]]","[Family-history-of-MI-OR-Family-history-of-myocardial-infarction, Family-history-of-thyroid-disorder, Chest-pressure, Intermittent-symptoms, Lightheaded, No-hair-changes-OR-no-nail-changes-OR-no-t...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
4,0,161,"17 y/o M , Dillon Cleveland comes with c/o of palpitaions. Palpitaions are sudden onset ond have been occuring for the last 3-4 months. He has had 1-2 episodes of palpitaions every month. The la...","[[father had acute MI], [Mother has thyroid condition], [pressure over chest], [1-2 episodes], [], [], [], [breathlessness], [], [palpitaions], [3-4 months], [17 y/o], [M]]","[[(601, 620)], [(571, 599)], [(258, 277)], [(149, 161)], [], [], [], [(226, 240)], [], [(46, 57)], [(125, 135)], [(0, 6)], [(7, 8)]]","[Family-history-of-MI-OR-Family-history-of-myocardial-infarction, Family-history-of-thyroid-disorder, Chest-press

# Folds
There are two possibilities that come to my mind for splitting the data : 
- A k-fold on features stratified by `case_num`
- A k-fold on features grouped by `case_num`

From my understanding, clinical cases will be the same in the train and test data, hence I'm going with the first option.

In [18]:
from sklearn.model_selection import StratifiedKFold

K = 5
SEED = 2222

skf = StratifiedKFold(n_splits=K, random_state=SEED, shuffle=True)
splits = list(skf.split(X=patient_notes, y=patient_notes['case_num']))

# map indices to case numbers
folds = np.zeros(len(patient_notes), dtype=int)
for i, (train_idx, val_idx) in enumerate(splits):
    folds[val_idx] = i
    df_val = patient_notes.iloc[val_idx]

patient_notes['fold'] = folds

baseline_train = patient_notes[['case_num', 'pn_num', 'fold']]
baseline_train.to_csv('baseline_train.csv', index=False)

baseline_train

,case_num,pn_num,fold
0,0,16,1
1,0,41,3
2,0,46,2
3,0,100,1
4,0,161,4
...,...,...,...
795,9,95066,3
796,9,95128,0
797,9,95228,0
798,9,95243,4


# Metric

From the [evaluation page](https://www.kaggle.com/c/nbme-score-clinical-patient-notes/overview/evaluation) :
- This competition is evaluated by a micro-averaged F1 score.
- We score each character index as:
 - TP if it is within both a ground-truth and a prediction,
 - FN if it is within a ground-truth but not a prediction, and,
 - FP if it is within a prediction but not a ground truth.
- Finally, we compute an overall F1 score from the TPs, FNs, and FPs aggregated across all instances.

In [19]:
from sklearn.metrics import f1_score

def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    
    return f1_score(truths, preds)

In [20]:
preds = [[0, 0, 1], [0, 0, 0]]
truths = [[0, 0, 1], [1, 0, 0]]

micro_f1(preds, truths)

0.6666666666666666

Now we need to convert predicted spans to binary arrays indcating whether each character is predicted of not.

In [21]:
def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    
    return binary

In [22]:
spans_to_binary([[0, 5], [10, 15]])

array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])

In [23]:
def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
        
    bin_preds = []
    bin_truths = []
    
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue

        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
        
    return micro_f1(bin_preds, bin_truths)

We generate spans from a train example.

In [24]:
spans = patient_notes['location'][0]
spans = [[list(np.array(s).astype(int)) for s in span] for span in spans if len(span)]

pred = spans
truth = [span[:2] for span in spans]

print(pred)
print(truth)

[[[696, 724]], [[668, 693]], [[203, 217]], [[70, 91], [176, 183]], [[222, 258]], [[321, 329], [404, 413], [652, 661]], [[26, 38], [96, 118]], [[56, 69]], [[5, 9]], [[10, 11]]]
[[[696, 724]], [[668, 693]], [[203, 217]], [[70, 91], [176, 183]], [[222, 258]], [[321, 329], [404, 413]], [[26, 38], [96, 118]], [[56, 69]], [[5, 9]], [[10, 11]]]


In [25]:
span_micro_f1(pred, truth)

0.9779951100244498

# Baseline

We basically perform string matching on all the data.

### Preparation

In [26]:
def location_to_span(location):
    spans = []
    for loc in location:
        if ";" in loc:
            loc = loc.split(';')
        else:
            loc = [loc]
        
        for l in loc:
            spans.append(list(np.array(l.split(' ')).astype(int)))
    
    return spans

In [27]:
df = train_data.copy()
patient_notes = pd.read_csv(DATA_PATH + "patient_notes.csv")

df = df.merge(patient_notes, how="left")

train_data


,id,case_num,pn_num,feature_num,annotation,location_raw,feature_text,pn_history,location
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myocardial-infarction,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(696, 724)]"
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(668, 693)]"
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(203, 217)]"
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(70, 91), (176, 183)]"
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(222, 258)]"
...,...,...,...,...,...,...,...,...,...
14295,95333_912,9,95333,912,[],[],Family-history-of-migraines,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...",[]
14296,95333_913,9,95333,913,[],[],Female,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...",[]
14297,95333_914,9,95333,914,[photobia],[274 282],Photophobia,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...","[(274, 282)]"
14298,95333_915,9,95333,915,[no sick contacts],[421 437],No-known-illness-contacts,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...","[(421, 437)]"


In [28]:
df_folds = pd.read_csv('folds.csv')
df = df.merge(df_folds, how="left", on=["case_num", "pn_num"])

In [37]:
df['span'] = df['location'] #.apply(location_to_span)
df

,id,case_num,pn_num,feature_num,annotation,location_raw,feature_text,pn_history,location,fold,span
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myocardial-infarction,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(696, 724)]",1,"[(696, 724)]"
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(668, 693)]",1,"[(668, 693)]"
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(203, 217)]",1,"[(203, 217)]"
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(70, 91), (176, 183)]",1,"[(70, 91), (176, 183)]"
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this ti...","[(222, 258)]",1,"[(222, 258)]"
...,...,...,...,...,...,...,...,...,...,...,...
11435,95333_912,9,95333,912,[],[],Family-history-of-migraines,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...",[],0,[]
11436,95333_913,9,95333,913,[],[],Female,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...",[],0,[]
11437,95333_914,9,95333,914,[photobia],[274 282],Photophobia,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...","[(274, 282)]",0,"[(274, 282)]"
11438,95333_915,9,95333,915,[no sick contacts],[421 437],No-known-illness-contacts,"Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no allev...","[(421, 437)]",0,"[(421, 437)]"


### Evaluation

In [38]:
    matching_dict = df_train[['case_num', 'feature_num', 'annotation']].groupby(['case_num', 'feature_num']).agg(list).T.to_dict()
    matching_dict = {k: np.concatenate(v['annotation']) for k, v in matching_dict.items()}
    matching_dict = {k: np.unique([v_.lower() for v_ in v]) for k, v in matching_dict.items()}
    matching_dict

{(0,
  0): array(['cad in father', 'dad - heart attack', 'dad had a heart attack',
        'dad had heart attack', 'dad had mi', 'dad has cardiac issues',
        'dad hx mi', 'dad mi', 'dad with "heart attack',
        'dad with recent heart attack', 'dad with recent heart attcak',
        'dad- heart attack', 'dad-mi', 'dad: heart problem', 'father - mi',
        'father had a heart attack', 'father had acute mi',
        'father had an mi', 'father had heart attack',
        'father had heart disease', 'father had mi',
        'father had possible mi', 'father had problems with heart',
        'father has heart problem', 'father heart attach',
        'father heart attack', 'father heart problem', 'father hx of mi',
        'father mi', 'father suffer an mi', 'father with herat attack',
        'father with mi', 'father- mi', 'father-mi',
        'father-recently had an mi', 'father: heart attack',
        'father: myocardial infarction',
        'heart attack 1 year ago - father', 

In [39]:
for fold in range(K):
    
    print(f"\n-------------   Fold {fold + 1} / {K}  -------------\n")
    
    df_train = df[df['fold'] != fold].reset_index(drop=True)
    df_val = df[df['fold'] == fold].reset_index(drop=True)
    
    matching_dict = df_train[['case_num', 'feature_num', 'annotation']].groupby(['case_num', 'feature_num']).agg(list).T.to_dict()
    matching_dict = {k: np.concatenate(v['annotation']) for k, v in matching_dict.items()}
    matching_dict = {k: np.unique([v_.lower() for v_ in v]) for k, v in matching_dict.items()}
    
    preds = []
    for i in range(len(df_val)):
        
        key = (df_val['case_num'][i], df_val['feature_num'][i])
#         print(key)
        
        candidates = matching_dict[key]
        
        text = df_val['pn_history'][i].lower()
        
        spans = []
        for c in candidates:
            start = text.find(c)
            if start > -1:
                spans.append([start, start + len(c)])
        
        preds.append(spans)
        
    score = span_micro_f1(preds, df_val['span'])

    print(f"-> F1 score: {score :.3f}")
    
#     break


-------------   Fold 1 / 5  -------------

-> F1 score: 0.567

-------------   Fold 2 / 5  -------------

-> F1 score: 0.580

-------------   Fold 3 / 5  -------------

-> F1 score: 0.580

-------------   Fold 4 / 5  -------------

-> F1 score: 0.579

-------------   Fold 5 / 5  -------------

-> F1 score: 0.559


### Inference

In [40]:
df_test = pd.read_csv(DATA_PATH + "test.csv")
df_test = df_test.merge(patient_notes, how="left")

In [41]:
df_train = train_data.copy()

# Cache matching dict with keys (case_num, feature_num) and values (list of annotations) of entire corpus
matching_dict = df_train[['case_num', 'feature_num', 'annotation']].groupby(['case_num', 'feature_num']).agg(list).T.to_dict()
matching_dict = {k: np.concatenate(v['annotation']) for k, v in matching_dict.items()}
matching_dict = {k: np.unique([v_.lower() for v_ in v]) for k, v in matching_dict.items()}

def predict(X, Y, Z):
    '''
    case number X, feature id Y, patient note Z
    '''
    candidates = matching_dict[(X, Y)]
    print(candidates)

    spans = []
    for c in candidates:
        start = Z.find(c)
        if start > -1:
            spans.append([start, start + len(c)])
    preds.append(spans)

    # list of predicted spans
    return preds_to_location(preds)

preds = []
for i in range(len(df_test)):
    print(df_test['case_num'][i], df_test['feature_num'][i], df_test['pn_history'][i])
    predict(df_test['case_num'][i], df_test['feature_num'][i], df_test['pn_history'][i])
    

0 0 HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. 
PMHx: none
Rx: uses friends adderrall
FHx: mom with "thyroid disease," dad with recent heart attcak
All: none
Immunizations: up to date
SHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms
['cad in father' 'dad - heart attack' 'dad had a heart attack'


### Submission

In [42]:
def preds_to_location(preds):
    locations = []
    for pred in preds:
        loc = ";".join([" ".join(np.array(p).astype(str)) for p in pred])
        locations.append(loc)
    return locations

In [43]:
sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')
sub['location'] = preds_to_location(preds)

sub.to_csv('submission.csv', index=False)
sub

,id,location
0,00016_000,696 724
1,00016_001,668 693
2,00016_002,203 217
3,00016_003,83 90;83 91;70 82;70 91
4,00016_004,222 258


*Thanks for reading !*